In [132]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix, balanced_accuracy_score, matthews_corrcoef
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import NearMiss
import scipy.sparse as sp
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import hstack

In [61]:
df = pd.read_csv("D:\\Private\\University\\Research\\Data Sets\\all_tickets.csv")
df

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4
...,...,...,...,...,...,...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july re git space for hi got righ...,0,4,21,1,39,1,3
48545,error,sent july error hi guys can you help out with ...,0,4,9,41,10,2,3
48546,connection issues,sent tuesday july connection issues hello have...,0,5,32,0,32,2,3
48547,error cube reports,sent tuesday july error hello guys have receiv...,0,4,21,41,35,1,3


In [62]:
nltk.download('stopwords')
nltk.download('punkt')
df['description'] = df['body']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
df = df.dropna(subset=['body', 'urgency'])

def preprosessor(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english') + list(punctuation))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

In [64]:
df['preprocessed_description'] = df['description'].apply(preprosessor)
df

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact,description,preprocessed_description
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4,hi since recruiter lead permission approve req...,hi since recruiter lead permission approve req...
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4,icon dear please setup icon per icon engineers...,icon dear please setup icon per icon engineers...
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4,work experience user hi work experience studen...,work experience user hi work experience studen...
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4,requesting meeting hi please help follow equip...,requesting meeting hi please help follow equip...
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4,re expire days hi ask help update passwords co...,expire days hi ask help update passwords colle...
...,...,...,...,...,...,...,...,...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july re git space for hi got righ...,0,4,21,1,39,1,3,sent tuesday july re git space for hi got righ...,sent tuesday july git space hi got rights prob...
48545,error,sent july error hi guys can you help out with ...,0,4,9,41,10,2,3,sent july error hi guys can you help out with ...,sent july error hi guys help error appearing w...
48546,connection issues,sent tuesday july connection issues hello have...,0,5,32,0,32,2,3,sent tuesday july connection issues hello have...,sent tuesday july connection issues hello conn...
48547,error cube reports,sent tuesday july error hello guys have receiv...,0,4,21,41,35,1,3,sent tuesday july error hello guys have receiv...,sent tuesday july error hello guys received er...


In [69]:
dfwt = df[['title','preprocessed_description', 'urgency']]
dfwt = dfwt.dropna(subset=['preprocessed_description', 'urgency', 'title'])
dfwt

,title,preprocessed_description,urgency
1,connection with icon,icon dear please setup icon per icon engineers...,3
2,work experience user,work experience user hi work experience studen...,3
3,requesting for meeting,requesting meeting hi please help follow equip...,3
4,reset passwords for external accounts,expire days hi ask help update passwords colle...,3
5,mail,verification warning hi got attached please ad...,3
...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july git space hi got rights prob...,1
48545,error,sent july error hi guys help error appearing w...,2
48546,connection issues,sent tuesday july connection issues hello conn...,2
48547,error cube reports,sent tuesday july error hello guys received er...,1


In [70]:
df = df[['preprocessed_description', 'urgency']]
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [71]:
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [72]:
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [73]:
unique_values = df['urgency'].value_counts()
unique_values

urgency
3    34621
1     6748
2     5528
0     1652
Name: count, dtype: int64

In [47]:
df

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact,description,preprocessed_description
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4,hi since recruiter lead permission approve req...,hi since recruiter lead permission approve req...
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4,icon dear please setup icon per icon engineers...,icon dear please setup icon per icon engineers...
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4,work experience user hi work experience studen...,work experience user hi work experience studen...
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4,requesting meeting hi please help follow equip...,requesting meeting hi please help follow equip...
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4,re expire days hi ask help update passwords co...,expire days hi ask help update passwords colle...
...,...,...,...,...,...,...,...,...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july re git space for hi got righ...,0,4,21,1,39,1,3,sent tuesday july re git space for hi got righ...,sent tuesday july git space hi got rights prob...
48545,error,sent july error hi guys can you help out with ...,0,4,9,41,10,2,3,sent july error hi guys can you help out with ...,sent july error hi guys help error appearing w...
48546,connection issues,sent tuesday july connection issues hello have...,0,5,32,0,32,2,3,sent tuesday july connection issues hello have...,sent tuesday july connection issues hello conn...
48547,error cube reports,sent tuesday july error hello guys have receiv...,0,4,21,41,35,1,3,sent tuesday july error hello guys have receiv...,sent tuesday july error hello guys received er...


In [12]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
# Fit TF-IDF vectorizer on the training data and transform both training and testing data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
# Define a broader parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],  # Explore a wider range of C values
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'linear']  # Consider both RBF and linear kernels
}
# Initialize an SVM classifier (kernel selection happens during grid search)
svm_classifier = SVC(class_weight={1: 2, 2: 3})  # Class weights (adjust if needed)
# Perform grid search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_tfidf, y_train)
# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)
# Train the classifier with the best hyperparameters on the entire training set
best_svm_classifier = SVC(**best_params)  # Unpack best parameters from grid search
best_svm_classifier.fit(X_train_tfidf, y_train)
# Make predictions on the testing data
y_pred = best_svm_classifier.predict(X_test_tfidf)
# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
# (Optional) Perform cross-validation for a more robust evaluation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(best_svm_classifier, X_tfidf, y, cv=5, scoring='accuracy')
print("Cross-validation accuracy:", cv_scores.mean())

KeyboardInterrupt: 

In [13]:
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [14]:
 # Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed

# Fit TF-IDF vectorizer on the training data and transform both training and testing data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Define a broader parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],  # Explore a wider range of C values
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'linear']  # Consider both RBF and linear kernels
}

# Initialize an SVM classifier (kernel selection happens during grid search)
svm_classifier = SVC(class_weight={1: 2, 2: 3})  # Class weights (adjust if needed)

# Perform grid search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)  # Utilize all available CPU cores
grid_search.fit(X_train_tfidf, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

# Train the classifier with the best hyperparameters on the entire training set
best_svm_classifier = SVC(**best_params)  # Unpack best parameters from grid search
best_svm_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = best_svm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# (Optional) Perform cross-validation for a more robust evaluation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(best_svm_classifier, X_train_tfidf, y_train, cv=5, scoring='accuracy', n_jobs=-1)  # Utilize all available CPU cores
print("Cross-validation accuracy:", cv_scores.mean())

Best hyperparameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Classification Report:
              precision    recall  f1-score   support

           1       0.69      0.83      0.75      1665
           2       0.67      0.37      0.48      1115
           3       0.98      1.00      0.99      6930

    accuracy                           0.90      9710
   macro avg       0.78      0.73      0.74      9710
weighted avg       0.89      0.90      0.89      9710

Accuracy: 0.8983522142121524
Cross-validation accuracy: 0.8956203143378924


In [15]:
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [16]:
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [18]:
X = df['preprocessed_description']
y = df['urgency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize Gradient Boosting Classifier
gbm_classifier = GradientBoostingClassifier()

# Train the classifier
gbm_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = gbm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.68      0.78      0.73      1665
           2       0.66      0.30      0.41      1115
           3       0.95      1.00      0.97      6930

    accuracy                           0.88      9710
   macro avg       0.76      0.69      0.70      9710
weighted avg       0.87      0.88      0.87      9710

Accuracy: 0.8816683831101957


In [25]:
X = df['preprocessed_description']
y = df['urgency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Define class weights (adjust as needed)
class_weights = {1: 5, 2: 10, 3: 1}  # Higher weight for urgency level 2

# Compute sample weights based on class labels
sample_weights = np.array([class_weights[label] for label in y_train])

# Initialize Gradient Boosting Classifier with class weights
gbm_classifier = GradientBoostingClassifier()

# Train the classifier
gbm_classifier.fit(X_train_tfidf, y_train, sample_weight=sample_weights)

# Make predictions on the testing data
y_pred = gbm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.74      0.52      0.61      1665
           2       0.41      0.71      0.52      1115
           3       0.99      0.94      0.97      6930

    accuracy                           0.84      9710
   macro avg       0.71      0.72      0.70      9710
weighted avg       0.88      0.84      0.85      9710

Accuracy: 0.84346035015448


In [12]:
X = df['preprocessed_description']
y = df['urgency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE to oversample the minority classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize Gradient Boosting Classifier
gbm_classifier = GradientBoostingClassifier()

# Train the classifier on the resampled training data
gbm_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = gbm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.72      0.67      0.69      1665
           2       0.57      0.48      0.52      1115
           3       0.96      0.99      0.97      6930

    accuracy                           0.88      9710
   macro avg       0.75      0.71      0.73      9710
weighted avg       0.87      0.88      0.87      9710

Accuracy: 0.8792996910401648
Balanced Accuracy: 0.7141723634997178

Confusion Matrix:
[[1113  372  180]
 [ 436  535  144]
 [   6   34 6890]]


In [13]:
df

,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [15]:
# Combine X_train and y_train into a DataFrame
train_data = pd.concat([X_train, y_train], axis=1)

# Identify the indices of samples belonging to class 3
class_3_indices = train_data[train_data['urgency'] == 3].index

# Specify the sampling strategy for class 3 to achieve the desired ratio
undersample_ratio = {1: len(y_train[y_train == 1]), 2: len(y_train[y_train == 2]), 3: len(class_3_indices) // 2}  # Undersample class 3 to half of its original size

# Initialize RandomUnderSampler with specified sampling strategy
undersampler = RandomUnderSampler(sampling_strategy=undersample_ratio, random_state=42)

# Perform undersampling on the training data
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_resampled).value_counts())

urgency
3    13845
1     6735
2     4413
Name: count, dtype: int64


In [16]:
gbm_classifier = GradientBoostingClassifier()

# Train the classifier on the resampled training data
gbm_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = gbm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.68      0.78      0.73      1665
           2       0.64      0.32      0.42      1115
           3       0.96      1.00      0.98      6930

    accuracy                           0.88      9710
   macro avg       0.76      0.70      0.71      9710
weighted avg       0.87      0.88      0.87      9710

Accuracy: 0.8837281153450052
Balanced Accuracy: 0.7000393906223502

Confusion Matrix:
[[1305  196  164]
 [ 618  354  143]
 [   1    7 6922]]


In [24]:
# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_resampled).value_counts())

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

urgency
3    27691
2    27691
1    27691
Name: count, dtype: int64


NameError: name 'XGBClassifier' is not defined

In [17]:
from xgboost import XGBClassifier

In [20]:
# Preprocess text data and split into features and target variable
X = df['preprocessed_description']
y = df['urgency'] - 1  # Adjust class labels to start from 0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_resampled).value_counts())

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

urgency
2    27691
1    27691
0    27691
Name: count, dtype: int64
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.73      0.72      1665
           1       0.59      0.51      0.54      1115
           2       0.98      1.00      0.99      6930

    accuracy                           0.89      9710
   macro avg       0.76      0.74      0.75      9710
weighted avg       0.89      0.89      0.89      9710

Accuracy: 0.894129763130793


In [21]:
# Preprocess text data and split into features and target variable
X = df['preprocessed_description']
y = df['urgency'] - 1  # Adjust class labels to start from 0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_resampled).value_counts())

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier(scale_pos_weight={0: 2, 1: 2})

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

urgency
2    27691
1    27691
0    27691
Name: count, dtype: int64


D:\Softwares\Jupyter\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:05:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.73      0.72      1665
           1       0.59      0.51      0.54      1115
           2       0.98      1.00      0.99      6930

    accuracy                           0.89      9710
   macro avg       0.76      0.74      0.75      9710
weighted avg       0.89      0.89      0.89      9710

Accuracy: 0.894129763130793


In [22]:
df


,preprocessed_description,urgency
0,hi since recruiter lead permission approve req...,3
1,icon dear please setup icon per icon engineers...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,expire days hi ask help update passwords colle...,3
...,...,...
48544,sent tuesday july git space hi got rights prob...,1
48545,sent july error hi guys help error appearing w...,2
48546,sent tuesday july connection issues hello conn...,2
48547,sent tuesday july error hello guys received er...,1


In [33]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency'] - 1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Get the number of samples in class 1
class_1_size = len(y_train[y_train == 1])

# Define the undersampling ratios for each class
undersampling_ratios = {
    0: len(y_train[y_train == 0]),  # Keep class 0 at original size
    1: class_1_size,  # Set class 1 to its original size
    2: class_1_size  # Set class 2 to the same size as class 1
}

# Initialize RandomUnderSampler with the specified ratios
undersampler = RandomUnderSampler(sampling_strategy=undersampling_ratios, random_state=42)

# Perform undersampling on the training data
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_tfidf, y_train)
print(pd.Series(y_train_resampled).value_counts())

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

urgency
0    6735
1    4413
2    4413
Name: count, dtype: int64
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.81      0.74      1665
           1       0.59      0.42      0.49      1115
           2       0.99      0.99      0.99      6930

    accuracy                           0.89      9710
   macro avg       0.75      0.74      0.74      9710
weighted avg       0.89      0.89      0.89      9710

Accuracy: 0.8912461380020598


In [34]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency'] - 1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Get the number of samples in class 0 and class 1
class_0_size = len(y_train[y_train == 0])
class_1_size = len(y_train[y_train == 1])

# Define the undersampling ratios for each class
undersampling_ratios = {
    0: int(0.5 * class_0_size),  # Reduce class 0 to 50% of original size
    1: class_1_size,  # Keep class 1 at original size
    2: class_1_size  # Set class 2 to the same size as class 1
}

# Initialize RandomUnderSampler with the specified ratios
undersampler = RandomUnderSampler(sampling_strategy=undersampling_ratios, random_state=42)

# Perform undersampling on the training data
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_tfidf, y_train)
print(pd.Series(y_train_resampled).value_counts())

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

urgency
1    4413
2    4413
0    3367
Name: count, dtype: int64
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.52      0.61      1665
           1       0.49      0.71      0.58      1115
           2       0.99      0.99      0.99      6930

    accuracy                           0.88      9710
   macro avg       0.74      0.74      0.73      9710
weighted avg       0.89      0.88      0.88      9710

Accuracy: 0.8754891864057672


In [43]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the training data
xgb_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.19      0.28       338
           1       0.58      0.68      0.63      1327
           2       0.58      0.48      0.52      1115
           3       0.98      1.00      0.99      6930

    accuracy                           0.87      9710
   macro avg       0.66      0.59      0.60      9710
weighted avg       0.86      0.87      0.86      9710

Accuracy: 0.8682801235839341


In [44]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.47      0.42       338
           1       0.60      0.57      0.59      1327
           2       0.60      0.51      0.55      1115
           3       0.98      1.00      0.99      6930

    accuracy                           0.87      9710
   macro avg       0.64      0.64      0.64      9710
weighted avg       0.86      0.87      0.86      9710

Accuracy: 0.8653964984552008


In [46]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Define class weights (highest weight for class 0, lower weights for classes 1, 2, and 3)
class_weights = {0: 3, 1: 2, 2: 1, 3: 1}

# Initialize XGBoost classifier with class weights
xgb_classifier = XGBClassifier(class_weight=class_weights)

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

D:\Softwares\Jupyter\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:36:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.47      0.42       338
           1       0.60      0.57      0.59      1327
           2       0.60      0.51      0.55      1115
           3       0.98      1.00      0.99      6930

    accuracy                           0.87      9710
   macro avg       0.64      0.64      0.64      9710
weighted avg       0.86      0.87      0.86      9710

Accuracy: 0.8653964984552008


In [47]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Calculate the class imbalance ratio
imbalance_ratio = (len(y_train) - sum(y_train)) / sum(y_train)

# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize XGBoost classifier with scale_pos_weight
xgb_classifier = XGBClassifier(scale_pos_weight=imbalance_ratio)

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred)))

SyntaxError: unmatched ')' (2515841855.py, line 34)

In [48]:

# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Calculate the class imbalance ratio
imbalance_ratio = (len(y_train) - sum(y_train)) / sum(y_train)

# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Initialize XGBoost classifier with scale_pos_weight
xgb_classifier = XGBClassifier(scale_pos_weight=imbalance_ratio)

# Train the classifier on the resampled training data
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

D:\Softwares\Jupyter\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:44:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.47      0.42       338
           1       0.60      0.57      0.59      1327
           2       0.60      0.51      0.55      1115
           3       0.98      1.00      0.99      6930

    accuracy                           0.87      9710
   macro avg       0.64      0.64      0.64      9710
weighted avg       0.86      0.87      0.86      9710

Accuracy: 0.8653964984552008


In [52]:
# Split the data into features (X) and target variable (y)
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y_train)

# Initialize SMOTE to perform oversampling
smote = SMOTE(random_state=42)

# Perform oversampling on the training data with class weights
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train, sample_weight=class_weights)

# Initialize XGBoost classifier with sample_weight
xgb_classifier = XGBClassifier()

# Train the classifier on the resampled training data with sample weights
xgb_classifier.fit(X_train_resampled, y_train_resampled, sample_weight=class_weights)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

TypeError: BaseSampler.fit_resample() got an unexpected keyword argument 'sample_weight'

In [14]:
X = df['preprocessed_description']
y = df['urgency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize NearMiss sampler with NearMiss-1 strategy
near_miss = NearMiss(version=1)

# Fit the NearMiss sampler to the training data and resample it
X_train_resampled, y_train_resampled = near_miss.fit_resample(X_train_tfidf, y_train)

# Initialize and train your classifier (e.g., Support Vector Machine)
svm_classifier = SVC(kernel='rbf', class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance
svm_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.12      0.70      0.20       338
           1       0.62      0.24      0.35      1327
           2       0.43      0.48      0.45      1115
           3       0.99      0.85      0.92      6930

    accuracy                           0.72      9710
   macro avg       0.54      0.57      0.48      9710
weighted avg       0.85      0.72      0.76      9710

Accuracy: 0.7194644696189495


In [15]:
X = df['preprocessed_description']
y = df['urgency']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train your classifier (SVM with RBF kernel) with adjusted class weights
svm_classifier = SVC(kernel='rbf', class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance
svm_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.32      0.37       338
           1       0.60      0.62      0.61      1327
           2       0.56      0.55      0.56      1115
           3       0.98      1.00      0.99      6930

    accuracy                           0.87      9710
   macro avg       0.65      0.62      0.63      9710
weighted avg       0.86      0.87      0.87      9710

Accuracy: 0.8697219361483007


In [81]:
# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (48549,)
Shape of y: (47837,)


In [88]:
print("Length of X_preprocessed_description:", len(X_preprocessed_description))
print("Length of X_title:", len(X_title))
print("Length of y:", len(y))

Length of X_preprocessed_description: 47837
Length of X_title: 47837
Length of y: 47837


In [85]:
# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Split the data into training and testing sets
X_train_pd, X_test_pd, X_train_title, X_test_title, y_train, y_test = train_test_split(X_preprocessed_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'preprocessed_description'
tfidf_vectorizer_pd = TfidfVectorizer(max_features=10000) 
X_train_tfidf_pd = tfidf_vectorizer_pd.fit_transform(X_train_pd)
X_test_tfidf_pd = tfidf_vectorizer_pd.transform(X_test_pd)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_pd, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_pd, X_test_tfidf_title))

# Initialize and train your classifier (SVM with RBF kernel) with adjusted class weights
svm_classifier = SVC(kernel='rbf', class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance
svm_classifier.fit(X_train_tfidf_combined, y_train)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf_combined)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.35      0.41       337
           1       0.63      0.67      0.65      1301
           2       0.59      0.58      0.58      1045
           3       0.99      0.99      0.99      6885

    accuracy                           0.88      9568
   macro avg       0.67      0.65      0.66      9568
weighted avg       0.88      0.88      0.88      9568

Accuracy: 0.8801212374581939


In [90]:
# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Split the data into training and testing sets
X_train_pd, X_test_pd, X_train_title, X_test_title, y_train, y_test = train_test_split(X_preprocessed_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'preprocessed_description'
tfidf_vectorizer_pd = TfidfVectorizer(max_features=10000) 
X_train_tfidf_pd = tfidf_vectorizer_pd.fit_transform(X_train_pd)
X_test_tfidf_pd = tfidf_vectorizer_pd.transform(X_test_pd)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_pd, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_pd, X_test_tfidf_title))

# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Split the data into training and testing sets
X_train_pd, X_test_pd, X_train_title, X_test_title, y_train, y_test = train_test_split(X_preprocessed_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'preprocessed_description'
tfidf_vectorizer_pd = TfidfVectorizer(max_features=10000) 
X_train_tfidf_pd = tfidf_vectorizer_pd.fit_transform(X_train_pd)
X_test_tfidf_pd = tfidf_vectorizer_pd.transform(X_test_pd)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_pd, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_pd, X_test_tfidf_title))

# Initialize and train your classifier (SVM with RBF kernel) with adjusted class weights
svm_classifier = SVC(kernel='rbf', class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance
svm_classifier.fit(X_train_tfidf_combined, y_train)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf_combined)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
svm_classifier.fit(X_train_tfidf_combined, y_train)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf_combined)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.37      0.41       337
           1       0.58      0.80      0.67      1301
           2       0.69      0.33      0.45      1045
           3       0.98      1.00      0.99      6885

    accuracy                           0.88      9568
   macro avg       0.68      0.62      0.63      9568
weighted avg       0.88      0.88      0.87      9568

Accuracy: 0.8754180602006689
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.37      0.41       337
           1       0.58      0.80      0.67      1301
           2       0.69      0.33      0.45      1045
           3       0.98      1.00      0.99      6885

    accuracy                           0.88      9568
   macro avg       0.68      0.62      0.63      9568
weighted avg       0.88      0.88      0.87      9568

Accuracy: 0.8754180602006689
Classification Report:
   

In [92]:
dfwt.loc[dfwt['urgency'] == 0, 'urgency'] = 1
dfwt

,title,preprocessed_description,urgency
1,connection with icon,icon dear please setup icon per icon engineers...,3
2,work experience user,work experience user hi work experience studen...,3
3,requesting for meeting,requesting meeting hi please help follow equip...,3
4,reset passwords for external accounts,expire days hi ask help update passwords colle...,3
5,mail,verification warning hi got attached please ad...,3
...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july git space hi got rights prob...,1
48545,error,sent july error hi guys help error appearing w...,2
48546,connection issues,sent tuesday july connection issues hello conn...,2
48547,error cube reports,sent tuesday july error hello guys received er...,1


In [93]:
dfwt.loc[dfwt['urgency'] == 0, 'urgency'] = 1

In [94]:
dfwt

,title,preprocessed_description,urgency
1,connection with icon,icon dear please setup icon per icon engineers...,3
2,work experience user,work experience user hi work experience studen...,3
3,requesting for meeting,requesting meeting hi please help follow equip...,3
4,reset passwords for external accounts,expire days hi ask help update passwords colle...,3
5,mail,verification warning hi got attached please ad...,3
...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july git space hi got rights prob...,1
48545,error,sent july error hi guys help error appearing w...,2
48546,connection issues,sent tuesday july connection issues hello conn...,2
48547,error cube reports,sent tuesday july error hello guys received er...,1


In [95]:
unique_values = dfwt['urgency'].value_counts()
unique_values

urgency
3    34033
1     8342
2     5462
Name: count, dtype: int64

In [130]:
# SMO classifier

# Assuming X contains your features and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Combine 'preprocessed_description' and 'title' into a single feature
X_combined = X_preprocessed_description + ' ' + X_title

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize SMO classifier
smo_classifier = SVC(kernel='linear', C=1.0, class_weight='balanced')

# Train the classifier on the training data
smo_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the testing data
y_pred = smo_classifier.predict(X_test_tfidf)
conf_matrix = confusion_matrix(y_test, y_pred)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(conf_matrix)

Classification Report:
              precision    recall  f1-score   support

           1       0.70      0.67      0.68      1638
           2       0.50      0.58      0.54      1045
           3       0.99      0.98      0.99      6885

    accuracy                           0.88      9568
   macro avg       0.73      0.74      0.74      9568
weighted avg       0.89      0.88      0.89      9568

Accuracy: 0.8834657190635451


In [131]:
# SVM with rbf and class weight balanced
# Current Best
# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Split the data into training and testing sets
X_train_pd, X_test_pd, X_train_title, X_test_title, y_train, y_test = train_test_split(X_preprocessed_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'preprocessed_description'
tfidf_vectorizer_pd = TfidfVectorizer(max_features=10000) 
X_train_tfidf_pd = tfidf_vectorizer_pd.fit_transform(X_train_pd)
X_test_tfidf_pd = tfidf_vectorizer_pd.transform(X_test_pd)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_pd, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_pd, X_test_tfidf_title))

# Initialize and train your classifier (SVM with RBF kernel) with adjusted class weights
svm_classifier = SVC(kernel='rbf', class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance
svm_classifier.fit(X_train_tfidf_combined, y_train)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf_combined)
conf_matrix = confusion_matrix(y_test, y_pred)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:")
print(conf_matrix)

Classification Report:
              precision    recall  f1-score   support

           1       0.73      0.77      0.75      1638
           2       0.62      0.55      0.58      1045
           3       0.99      0.99      0.99      6885

    accuracy                           0.91      9568
   macro avg       0.78      0.77      0.77      9568
weighted avg       0.90      0.91      0.90      9568

Accuracy: 0.9061454849498328
Confusion Matrix:
[[1266  331   41]
 [ 434  572   39]
 [  26   27 6832]]


In [ ]:
# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Split the data into training and testing sets
X_train_pd, X_test_pd, X_train_title, X_test_title, y_train, y_test = train_test_split(X_preprocessed_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'preprocessed_description'
tfidf_vectorizer_pd = TfidfVectorizer(max_features=10000) 
X_train_tfidf_pd = tfidf_vectorizer_pd.fit_transform(X_train_pd)
X_test_tfidf_pd = tfidf_vectorizer_pd.transform(X_test_pd)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_pd, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_pd, X_test_tfidf_title))

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf_combined, y_train)

# Initialize and train your classifier (SVM with RBF kernel) with adjusted class weights
svm_classifier = SVC(kernel='rbf')  # You can remove class_weight='balanced' when using SMOTE
svm_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_test_tfidf_combined)
conf_matrix = confusion_matrix(y_test, y_pred)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
dfwt

In [ ]:
# Assuming X contains your features and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Initialize TF-IDF vectorizer for preprocessed_description
tfidf_vectorizer_desc = TfidfVectorizer(max_features=10000) 
X_train_tfidf_desc = tfidf_vectorizer_desc.fit_transform(X_preprocessed_description)

# Initialize TF-IDF vectorizer for title
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_title)

# Concatenate TF-IDF vectors for both preprocessed_description and title
X_train_tfidf_combined = hstack([X_train_tfidf_desc, X_train_tfidf_title])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf_combined, y, test_size=0.2, random_state=42)

# Initialize Naive Bayes classifier
nb_classifier = MultinomialNB(alpha=2)

# Train the classifier on the training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = nb_classifier.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(conf_matrix)

In [119]:
dfwt.drop_duplicates()

,title,preprocessed_description,urgency
1,connection with icon,icon dear please setup icon per icon engineers...,3
2,work experience user,work experience user hi work experience studen...,3
3,requesting for meeting,requesting meeting hi please help follow equip...,3
4,reset passwords for external accounts,expire days hi ask help update passwords colle...,3
5,mail,verification warning hi got attached please ad...,3
...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july git space hi got rights prob...,1
48545,error,sent july error hi guys help error appearing w...,2
48546,connection issues,sent tuesday july connection issues hello conn...,2
48547,error cube reports,sent tuesday july error hello guys received er...,1


In [122]:
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Initialize TF-IDF vectorizer for preprocessed_description
tfidf_vectorizer_desc = TfidfVectorizer(max_features=10000)
X_train_tfidf_desc = tfidf_vectorizer_desc.fit_transform(X_preprocessed_description)

# Initialize TF-IDF vectorizer for title
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000)
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_title)

# Apply RandomUnderSampler before feature concatenation (assuming class imbalance)
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled_desc, y_resampled = rus.fit_resample(X_train_tfidf_desc, y)

# Filter titles based on description indices kept after undersampling
description_indices_kept = rus.sample_indices_
X_title_filtered = X_title.iloc[description_indices_kept]

# Transform filtered titles using the title TF-IDF vectorizer
X_resampled_title = tfidf_vectorizer_title.transform(X_title_filtered)

# Concatenate TF-IDF vectors for descriptions and filtered titles
X_train_combined = hstack([X_resampled_desc, X_resampled_title])

# Train-Test Split on the combined resampled features
X_train, X_test, y_train, y_test = train_test_split(X_train_combined, y_resampled, test_size=0.2, random_state=42)

# Initialize Naive Bayes classifier
nb_classifier = MultinomialNB(alpha=2)  # Adjust alpha for smoothing (optional)

# Train the classifier on the undersampled training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = nb_classifier.predict(X_test)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.58      0.72      0.64      1098
           2       0.62      0.52      0.57      1113
           3       0.97      0.88      0.92      1067

    accuracy                           0.71      3278
   macro avg       0.72      0.71      0.71      3278
weighted avg       0.72      0.71      0.71      3278

Accuracy: 0.7068334350213545


In [128]:
# Assuming X contains your features and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Combine 'preprocessed_description' and 'title' into a single feature
X_combined = X_preprocessed_description + ' ' + X_title

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000) 
X_tfidf = tfidf_vectorizer.fit_transform(X_combined)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize Naive Bayes classifier
nb_classifier = MultinomialNB(alpha=0.01)  # Adjust alpha for smoothing (optional)

# Train the classifier on the training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = nb_classifier.predict(X_test)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.77      0.70      1638
           2       0.55      0.30      0.39      1045
           3       0.96      0.98      0.97      6885

    accuracy                           0.87      9568
   macro avg       0.72      0.68      0.68      9568
weighted avg       0.86      0.87      0.86      9568

Accuracy: 0.8681020066889632


In [133]:
# Assuming X contains your 'preprocessed_description' and 'title' fields separately and y contains your target variable
X_preprocessed_description = dfwt['preprocessed_description']
X_title = dfwt['title']
y = dfwt['urgency']

# Split the data into training and testing sets
X_train_pd, X_test_pd, X_train_title, X_test_title, y_train, y_test = train_test_split(X_preprocessed_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'preprocessed_description'
tfidf_vectorizer_pd = TfidfVectorizer(max_features=10000) 
X_train_tfidf_pd = tfidf_vectorizer_pd.fit_transform(X_train_pd)
X_test_tfidf_pd = tfidf_vectorizer_pd.transform(X_test_pd)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_pd, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_pd, X_test_tfidf_title))

# Initialize and train your Random Forest classifier
rf_classifier = RandomForestClassifier(class_weight='balanced', random_state=42)  # Use class_weight='balanced' to handle class imbalance
rf_classifier.fit(X_train_tfidf_combined, y_train)

# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test_tfidf_combined)
conf_matrix = confusion_matrix(y_test, y_pred)

# Evaluate the performance of the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:")
print(conf_matrix)

Classification Report:
              precision    recall  f1-score   support

           1       0.68      0.89      0.77      1638
           2       0.67      0.27      0.38      1045
           3       0.98      1.00      0.99      6885

    accuracy                           0.90      9568
   macro avg       0.78      0.72      0.71      9568
weighted avg       0.90      0.90      0.89      9568

Accuracy: 0.8995610367892977
Confusion Matrix:
[[1455  132   51]
 [ 692  280   73]
 [   7    6 6872]]
